In [1]:
import sympy as sym
import numpy as np
from math import e
from math import sqrt

x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

def GetHermiteRecursive(n,x):

    if n==0:
        poly = sym.Number(1)
    elif n==1:
        poly = 2*x
    else:
        poly = (2*x*GetHermiteRecursive(n-1,x)-2*(n-1)*GetHermiteRecursive(n-2,x))
   
    return sym.expand(poly,x)


In [2]:
GetHermiteRecursive(3,x)

8*x**3 - 12*x

In [3]:
def GetDHermite(n,x):
    Pn = GetHermiteRecursive(n,x)
    return sym.diff(Pn,x,1)


def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn
    
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)

        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

In [6]:
def GetAllRootsGHer(n):

    xn = np.linspace(-sqrt(4*n+1),sqrt(4*n+1),100)
    
    Hermite = []
    DHermite = []
    
    for i in range(n+1):
        Hermite.append(GetHermiteRecursive(i,x))
        DHermite.append(GetDHermite(i,x))
    
    poly = sym.lambdify([x],Hermite[n],'numpy')
    Dpoly = sym.lambdify([x],DHermite[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

In [7]:
GetAllRootsGHer(3)

array([-1.22474487,  0.        ,  1.22474487])

In [12]:
def GetWeightsGHer(n):

    Roots = GetAllRootsGHer(n)

    Weights = []


    for valor in range(n):
        HerRoot = GetHermiteRecursive(n-1,Roots[valor])
        weight = (2**(n-1)*np.math.factorial(n)*sqrt(np.pi))/((n**2)*HerRoot**2)
        Weights.append(weight)


    return Weights

In [13]:
GetWeightsGHer(3)

[0.295408975138747, 1.18163590060368, 0.295408975138747]

In [14]:
raices = GetAllRootsGHer(5)
pesos = GetWeightsGHer(5)
funcion = lambda x: x**4
I = 0
for i in range(5):
    I += pesos[i]*funcion(raices[i])

In [15]:
I

1.32934038799796